
how to scan sqlite tools?





how to scan chat logs?




In [ ]:

//import {fileURLToPath} from "url";
const fs = require('fs')
const path = require('path')
const importNotebook = importer.import
const {askLlamaAboutEmotions} = importNotebook('ask llm about emotions')
const {storeChatHistory} = importNotebook('how to cache chat logs')
//const { chatCache } = importer.import('cache chat history with llm descriptions')
const { askLlamaAboutConversation, askLlamaAboutCategory } = importNotebook('ask llm about chat conversations')
const llama = importNotebook('create llm session')

const hashCode = s => s.split('').reduce((a,b) => (((a << 5) - a) + b.charCodeAt(0))|0, 0)

const CHAT_DIRECTORIES = [
  '/Volumes/External/Personal/Collections/conversations',
  '/Volumes/External/Personal/Collections/conversations/Trillian/logs/AIM/Query',
  '/Volumes/External/Personal/Collections/conversations/Trillian/logs/MSN/Query',
  '/Volumes/External/Personal/Collections/conversations/Trillian/logs/old logs/AIM',
  '/Volumes/External/Personal/Collections/conversations/Trillian/logs/old logs/MSN',
  '/Volumes/External/Personal/Collections/conversations/Trillian/logs/older logs/AIM',
  '/Volumes/External/Personal/Collections/conversations/Trillian/logs/older logs/MSN',
]

const CHAT_DATES = [
  /(January|February|March|April|May|June|July|August|September|October|November|December) (\d{1,2}), \d{4}/gi,
  /(0[1-9]|[12]\d|3[01])\.(0[1-9]|1[0-2])\.\d{4}/gi,
  /(0[1-9]|1[0-2])\/(0[1-9]|[12]\d|3[01])\/\d{4}/gi,
  /\d{4}-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01])/gi,
  /(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (\d{1,2}), \d{4}/gi
]

const CHAT_TIMES = [
  /(0[1-9]|1[0-2]):([0-5]\d):([0-5]\d)\s?(AM|PM)/gi,
  /([01]\d|2[0-3]):([0-5]\d):([0-5]\d)/gi,
  /(0[1-9]|1[0-2]):([0-5]\d)\s?(AM|PM)/gi,
  /([01]\d|2[0-3]):([0-5]\d)/gi
]

const CHAT_PATH = path.join(__dirname, '..', 'Resources', 'Projects', 'conversations')

function cellNeedsTidying(cellId, chatCache) {
  return typeof chatCache[cellId] == 'undefined'
  || typeof chatCache[cellId].emotions == 'undefined'
  || typeof chatCache[cellId].category != 'string'
  || typeof chatCache[cellId].date == 'undefined'
  || !chatCache[cellId].date
  || chatCache[cellId].summary.match(/Find the derivative/gi)
  || chatCache[cellId].category == '<h1>'
  || typeof chatCache[cellId].from == 'undefined' || typeof chatCache[cellId].to == 'undefined'
}

function convertMessagesToNoDates(currentMessages) {
  let messageNoTimes = currentMessages.join('\n').trim()
  for(let i = 0; i < CHAT_DATES.length; i++) {
    messageNoTimes = messageNoTimes.replace(CHAT_DATES[i], '')
  }
  for(let i = 0; i < CHAT_TIMES.length; i++) {
    messageNoTimes = messageNoTimes.replace(CHAT_TIMES[i], '')
  }
  messageNoTimes = messageNoTimes.replace(/\[\s*\]\s*/gi, '')
  return messageNoTimes.split('\n')
}

async function askLlamaAboutChatLog(chatLog) {
  const {llmPrompt} = (await llama)

  if(!chatLog) {
    chatLog = 'fuji897@hotmail.com'
  }

  let chatHistory = []
  let chatPath
  for(let i = 0; i < CHAT_DIRECTORIES.length; i++) {
    chatPath = path.join(CHAT_DIRECTORIES[i], chatLog + '.log')
    if(fs.existsSync(chatPath)) {
      chatHistory = fs.readFileSync(chatPath).toLocaleString('utf-8').split('\n')
      break
    } else {
      chatPath = void 0
    }
  }
  if(!chatPath) return

  let chatCacheFile = path.join(CHAT_PATH, path.basename(chatPath).replace('.log', '') + '.json')
  let chatCache = {}
  if(fs.existsSync(chatCacheFile))
    chatCache = JSON.parse(fs.readFileSync(chatCacheFile).toString('utf-8'))

  let from = 0
  let mtime = fs.statSync(chatPath).mtime.getTime()
  let previousTime = mtime
  let cellCounter = 0
  let currentMessages = []
  let currentTotal = 0
  let cellId = chatPath + '[' + cellCounter + ']'
  for(let i = 0; i < chatHistory.length; i++) {
    if(currentMessages.length > 0 && currentTotal + chatHistory[i].length > 2048 || currentMessages.length == 20) {
      // process now
      // TODO: store result
      let messageBlock = currentMessages.join('\n')
      let messageNoTimes = convertMessagesToNoDates(currentMessages)
      let date = CHAT_DATES.map(d => (messageBlock.match(d) || [])[0]).filter(d=>d)[0]
      let time = CHAT_TIMES.map(d => (messageBlock.match(d) || [])[0]).filter(d=>d)[0]
      let now = new Date(date + ' ' + time).getTime() || previousTime
      if(date && time)
        previousTime = now
      let hash = hashCode(messageBlock)
      if(cellNeedsTidying(cellId, chatCache) || chatCache[cellId].hash != hash) {
        let summary = await askLlamaAboutConversation(messageNoTimes)
        let emotions = await askLlamaAboutEmotions(messageNoTimes)
        let category = await askLlamaAboutCategory(messageNoTimes)
        storeChatHistory(cellId, mtime, summary, emotions, category, hash, now, from, i - 1)
      }
      currentMessages = []
      currentTotal = 0
      from = i
      cellCounter++
      cellId = chatPath + '[' + cellCounter + ']'
    }

    if(chatHistory[i].trim().length == 0) continue

    currentMessages[currentMessages.length] = chatHistory[i]
    currentTotal += chatHistory[i].length
  }
  if(currentMessages.length > 0) {
    // process now
    let messageBlock = currentMessages.join('\n')
    let messageNoTimes = convertMessagesToNoDates(currentMessages)
    let date = CHAT_DATES.map(d => (messageBlock.match(d) || [])[0]).filter(d=>d)[0]
    let time = CHAT_TIMES.map(d => (messageBlock.match(d) || [])[0]).filter(d=>d)[0]
    let now = new Date(date + ' ' + time).getTime()
    let hash = hashCode(messageBlock)
    if(cellNeedsTidying(cellId, chatCache) || chatCache[cellId].hash != hash) {
      let summary = await askLlamaAboutConversation(messageNoTimes)
      let emotions = await askLlamaAboutEmotions(messageNoTimes)
      let category = await askLlamaAboutCategory(messageNoTimes)
      storeChatHistory(cellId, mtime, summary, emotions, category, hash, now, from, i - 1)
    }
    // TODO: store result
  }

  
}

module.exports = {
  askLlamaAboutChatLog,
  askLlamaAboutConversation
}



ask llm about emotions?



In [ ]:
const llama = importer.import('create llm session')

const EMOTIONS = [
  'Joyful',
  'Romantic',
  'Angry',
  'Confused',
  'Supportive',
  'Excited',
  'Nostalgic',
  'Grateful',
  'Sad',
  'Humorous',
  'Anxious',
  'Curious',
  'Inspired',
  'Defensive',
  'Assertive',
  'Empathetic',
  'Reflective',
  'Playful',
  'Hopeful',
  'Apologetic',
  'Lonely',
  'Proud',
  'Vulnerable',
  'Determined',
  'Aroused',
  'Neutral',
  'Dissident',
  'Rebelious',
  'Frustrated',
  'Helpful',
  'Enthusiastic',
  'Casual',
  'Enthusiasm',
  'Annoyed',
  'Touched',
  'Apologetic',
  'Regret',
  'Regretful',
  'Content',
  'Insecure',
  'Concerned',
  'Erotic',
  'Vulgar',
  'Bored',
  'Confused',
  'Caring',
  'Hesitant',
  'Flirty',
  'Flirtatious',
  'Eager',
  'Attentive',
  'Affectionate',
  'Charming',
  'Confident',
  'Smug',
  'Embarrassed',
  'Nervous',
  'Thoughtful',
  'Apprehensive',
  'Tired',
  'Amused',
  'Flustered',
  'Exasperated',
  'Longing',
  'Happy',
  'Uncertain',
  'Understanding',
  'Encouraging',
  'Upset',
  'Worried',
  'Self-Conscious',
  'Sarcastic',
  'Teasing',
  'Competitive',
  'Serious',
  'Impressed',
  'Amazed',
  'Suggestive',
  'Needy',
  'Relatable',
  'Sexual',
  'Horny',
  'Flattered',
  'Intrigued',
  'Lighthearted',
  'Relieved',
  'Protective',
  'Apathetic',
  'Distracted',
  'Reassured',
  'Detached',
  'Numb',
  'Optimistic',
  'Passionate',
  'Amusing',
  'Dismissive',
  'Disappointed',
  'Resigned',
  'Sympathetic',
  'Open',
  'Disgruntled',
  'Guilty',
  'Awkward',
  'Knowledgeable',
  'Disengaged',
  'Interested',
  'Surprised',
  'Downcast',
  'Observant',
  'Stressful',
  'Hurt',
  'Self-Doubt',
  'Overwhelmed',
  'Yearning',
  'Desireous',
  'Loving',
  'Despondent',
  'Unheard',
  'Hopeless',
  'Remorseful',
  'Lewd',
  'Amusement',
  'Critical',
  'Lust',
  'Manipulative',
  'Matter-of-fact',
  'Task-Oriented',
  'Transitional',
  'Impatient',
  'Negative',
  'Dissatisfied',
  'Indecisive',
  'Preference',
  'Assertiveness',
  'Anger',
  'Contempt',
  'Disdain',
  'Threat',
  'Apathy',
  'Condescending',
  'Disbelief',
  'Reassurance',
  'Determination',
  'Informative',
  'Dismissiveness',
  'Resignation',
  'Businesslike',
  'Cooperative',
  'Skeptical',
  'Shock',
  'Infatuation',
  'Friendly',
  'Indifferent',
]

const EMOTION_HEX = [
  '#FFFF00',
  '#FFC0CB',
  '#FF0000',
  '#D8BFD8',
  '#90EE90',
  '#FFA500',
  '#4682B4',
  '#FFD700',
  '#00008B',
  '#FFFF99',
  '#800080',
  '#40E0D0',
  '#FFC107',
  '#800000',
  '#FF4500',
  '#E6E6FA',
  '#B0C4DE',
  '#00FFFF',
  '#FFFACD',
  '#B0E0E6',
  '#191970',
  '#FFD700',
  '#FFE4E1',
  '#DC143C',
  '#FF4500',
  '#B0B0B0',
  '#556B2F',
  '#000000',
  '#800020',
  '#87CEEB',
  '#FFA500',
  '#5B84B1',
  '#FFA500',
  '#900020'
]

async function askLlamaAboutEmotions(currentMessages, session) {
  const {llmPrompt} = (await llama)
  if(typeof currentMessages == 'string') {
    currentMessages = [currentMessages]
  }

  let q1 = 'Can you derive the emotional contexts of this conversation:\n' + 
  currentMessages.join('\n') + '\nOnly give the emotions in the response.'
  console.log("User: " + q1);
  const a1 = await llmPrompt(q1);
  console.log("AI: " + a1);

  let q2 = 'Based on this description which emotions best fit:\n' + 
    a1 + '\nOut of this list of emotions which one is the closest match:' +
    EMOTIONS.join(', ') + '\nOnly respond with the matching emotions.'
  console.log("User: " + q2);
  const a2 = await llmPrompt(q2);
  console.log("AI: " + a2);
  let emotions = a2.trim().split(/\s*\n\s*|,\s*|\s*- |\s*\*+\s*/gi)

  console.log(emotions)
  return EMOTIONS.filter(e => emotions.includes(e) || a2.includes(e) || a1.includes(e))
}

module.exports = {
  askLlamaAboutEmotions,
  EMOTIONS,
  EMOTION_HEX
}



how to cache chat logs?



In [ ]:
const fs = require('fs')
const path = require('path')
const CHAT_PATH = path.join(__dirname, '..', 'Resources', 'Projects', 'conversations')
const { chatCache } = importer.import('cache chat history with llm descriptions')
const { updateCode } = importer.import('update code cell')

function storeChatHistory (cellId, mtime, summary, emotions, category, hash, date, from, to) {
  let chatCacheFile = path.join(CHAT_PATH, path.basename(cellId).replace(/.log\[[0-9]*\]/, '') + '.json')
  let chatCache = {}
  if(fs.existsSync(chatCacheFile))
    chatCache = JSON.parse(fs.readFileSync(chatCacheFile).toString('utf-8'))
  chatCache[cellId] = {
    mtime,
    summary,
    emotions,
    category,
    hash,
    date,
    from,
    to
  }
  fs.writeFileSync(chatCacheFile, JSON.stringify(chatCache, null, 4))
}

module.exports = {
  storeChatHistory
}



ask llm about chat conversations?


In [ ]:
const llama = importer.import('create llm session')

async function askLlamaAboutConversation(currentMessages) {
  const {llmPrompt} = (await llama)
  debugger
  let q1 = 'Can you summarize in two sentences what this conversation is about:\n' + 
  currentMessages.join('\n') + '\nPlease discard any pleasantries, documentation only.'
  console.log("User: " + q1);
  const a1 = await llmPrompt(q1);
  console.log("AI: " + a1);
  return a1.trim()
}

async function askLlamaAboutCategory(currentMessages) {
  const {llmPrompt} = (await llama)
  let q1 = 'Categorize this conversation in two or three words:\n' + 
  currentMessages.join('\n') + '\nOnly respond with the category.'
  console.log("User: " + q1);
  const a1 = await llmPrompt(q1);
  console.log("AI: " + a1);
  return a1.trim().split(/\s*\n\s*|,\s*|\s*- |\s*\* /gi)[0]
}

module.exports = {
  askLlamaAboutConversation,
  askLlamaAboutCategory
}




how to scan code history?







how to scan calendar journal entries?

how to ask if my theories are correct?


